In [0]:
%run "./goldcommonutilities"

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from delta.tables import DeltaTable

df_price = spark.read.table("silver.da_sales_dev.price")
df_sales = spark.read.table("silver.da_sales_dev.sales")
df = df_price.join(df_sales, (df_price['FRUIT_ID'] == df_sales['FRUIT_ID'])\
                             & (df_sales['SALES_TS'] >= df_price['START_TS'])\
                             & (df_sales['SALES_TS'] <= df_price['END_TS']), 'inner')
df = df.select( df_price["FRUIT_ID"],"FRUIT_NAME", "FRUIT_PRICE",\
                         "START_TS", "END_TS", "FRUITS_QTYAMOUNT", "SALES_TS")
df = df.withColumn("AMOUNT",col("FRUIT_PRICE")*col("FRUITS_QTYAMOUNT").cast("float"))
mergetotarget(df, ["FRUIT_ID","SALES_TS"],"gold.da_sales_dev.fruitsales")

In [0]:
df = spark.read.table("gold.da_sales_dev.fruitsales")
df.display()